# TV Script Generation

In this project, you'll generate your own [Seinfeld](https://en.wikipedia.org/wiki/Seinfeld) TV scripts using RNNs.  You'll be using part of the [Seinfeld dataset](https://www.kaggle.com/thec03u5/seinfeld-chronicles#scripts.csv) of scripts from 9 seasons.  The Neural Network you'll build will generate a new ,"fake" TV script, based on patterns it recognizes in this training data.

## Get the Data

The data is already provided for you in `./data/Seinfeld_Scripts.txt` and you're encouraged to open that file and look at the text. 
>* As a first step, we'll load in this data and look at some samples. 
* Then, you'll be tasked with defining and training an RNN to generate a new script!

In [6]:
import signal

from contextlib import contextmanager

import requests


DELAY = INTERVAL = 4 * 60  # interval time in seconds
MIN_DELAY = MIN_INTERVAL = 2 * 60
KEEPALIVE_URL = "https://nebula.udacity.com/api/v1/remote/keep-alive"
TOKEN_URL = "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token"
TOKEN_HEADERS = {"Metadata-Flavor":"Google"}


def _request_handler(headers):
    def _handler(signum, frame):
        requests.request("POST", KEEPALIVE_URL, headers=headers)
    return _handler


@contextmanager
def active_session(delay=DELAY, interval=INTERVAL):
    """
    Example:

    from workspace_utils import active session

    with active_session():
        # do long-running work here
    """
    token = requests.request("GET", TOKEN_URL, headers=TOKEN_HEADERS).text
    headers = {'Authorization': "STAR " + token}
    delay = max(delay, MIN_DELAY)
    interval = max(interval, MIN_INTERVAL)
    original_handler = signal.getsignal(signal.SIGALRM)
    try:
        signal.signal(signal.SIGALRM, _request_handler(headers))
        signal.setitimer(signal.ITIMER_REAL, delay, interval)
        yield
    finally:
        signal.signal(signal.SIGALRM, original_handler)
        signal.setitimer(signal.ITIMER_REAL, 0)


def keep_awake(iterable, delay=DELAY, interval=INTERVAL):
    """
    Example:

    from workspace_utils import keep_awake

    for i in keep_awake(range(5)):
        # do iteration with lots of work here
    """
    with active_session(delay, interval): yield from iterable

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# load in data
import helper
data_dir = './data/Seinfeld_Scripts.txt'
text = helper.load_data(data_dir)

## Explore the Data
Play around with `view_line_range` to view different parts of the data. This will give you a sense of the data you'll be working with. You can see, for example, that it is all lowercase text, and each new line of dialogue is separated by a newline character `\n`.

In [8]:
view_line_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))

lines = text.split('\n')
print('Number of lines: {}'.format(len(lines)))
word_count_line = [len(line.split()) for line in lines]
print('Average number of words in each line: {}'.format(np.average(word_count_line)))

print()
print('The lines {} to {}:'.format(*view_line_range))
print('\n'.join(text.split('\n')[view_line_range[0]:view_line_range[1]]))

Dataset Stats
Roughly the number of unique words: 46367
Number of lines: 109233
Average number of words in each line: 5.544240293684143

The lines 0 to 10:
jerry: do you know what this is all about? do you know, why were here? to be out, this is out...and out is one of the single most enjoyable experiences of life. people...did you ever hear people talking about we should go out? this is what theyre talking about...this whole thing, were all out now, no one is home. not one person here is home, were all out! there are people trying to find us, they dont know where we are. (on an imaginary phone) did you ring?, i cant find him. where did he go? he didnt tell me where he was going. he must have gone out. you wanna go out you get ready, you pick out the clothes, right? you take the shower, you get all ready, get the cash, get your friends, the car, the spot, the reservation...then youre standing around, what do you do? you go we gotta be getting back. once youre out, you wanna get back! y

---
## Implement Pre-processing Functions
The first thing to do to any dataset is pre-processing.  Implement the following pre-processing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following **tuple** `(vocab_to_int, int_to_vocab)`

In [9]:
words=tuple(set(text))
int_to_vocab=dict(enumerate(words))
print(int_to_vocab)

vocab_to_int={ch: ind for ind,ch in int_to_vocab.items()}
print(vocab_to_int)

{0: '{', 1: ' ', 2: '6', 3: 'm', 4: '>', 5: 'x', 6: '4', 7: 't', 8: '~', 9: ':', 10: '-', 11: '\x94', 12: '\x92', 13: 'f', 14: '7', 15: ';', 16: '!', 17: '5', 18: '$', 19: 'w', 20: '+', 21: 'k', 22: 'n', 23: '/', 24: ']', 25: '<', 26: '}', 27: ')', 28: 'r', 29: 'h', 30: ',', 31: '#', 32: '*', 33: 'p', 34: "'", 35: 'i', 36: '_', 37: 'a', 38: 'l', 39: 'b', 40: '\x93', 41: 'g', 42: 'c', 43: 'v', 44: '\n', 45: '&', 46: 'd', 47: 'j', 48: '"', 49: '%', 50: '.', 51: '8', 52: '0', 53: 'u', 54: 's', 55: '…', 56: 'y', 57: '?', 58: 'z', 59: '3', 60: '[', 61: '`', 62: '9', 63: 'e', 64: '(', 65: '=', 66: '1', 67: '2', 68: 'o', 69: '\\', 70: '\x91', 71: 'q'}
{'{': 0, ' ': 1, '6': 2, 'm': 3, '>': 4, 'x': 5, '4': 6, 't': 7, '~': 8, ':': 9, '-': 10, '\x94': 11, '\x92': 12, 'f': 13, '7': 14, ';': 15, '!': 16, '5': 17, '$': 18, 'w': 19, '+': 20, 'k': 21, 'n': 22, '/': 23, ']': 24, '<': 25, '}': 26, ')': 27, 'r': 28, 'h': 29, ',': 30, '#': 31, '*': 32, 'p': 33, "'": 34, 'i': 35, '_': 36, 'a': 37, 'l': 38,

In [10]:
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO 1: Implement Function
    words=set(text)
    #words_counts = Counter(text)
    #print(words_counts)
    
    #sorted_words = sorted(words_counts, key=words_counts.get, reverse=True)
    vocab_to_int = { word:idx for idx , word in enumerate(words, start=0) }
    int_to_vocab = { idx:word for idx , word in enumerate(words, start=0) }

    return (vocab_to_int,int_to_vocab)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks can create multiple ids for the same word. For example, "bye" and "bye!" would generate two different word ids.

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( **.** )
- Comma ( **,** )
- Quotation Mark ( **"** )
- Semicolon ( **;** )
- Exclamation mark ( **!** )
- Question mark ( **?** )
- Left Parentheses ( **(** )
- Right Parentheses ( **)** )
- Dash ( **-** )
- Return ( **\n** )

This dictionary will be used to tokenize the symbols and add the delimiter (space) around it.  This separates each symbols as its own word, making it easier for the neural network to predict the next word. Make sure you don't use a value that could be confused as a word; for example, instead of using the value "dash", try using something like "||dash||".

In [11]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenized dictionary where the key is the punctuation and the value is the token
    """
    # TODO 2: Implement Function
    list_symbols=['.',
                  '?',
                  ',',
                  '(',')',
                  '"',
                  ';',
                  '-',
                  '!',
                  '\n']
    
    list_tokens=['||PERIOD||',
                 '||QUESTION_MARK||',
                 '||COMMA||',
                 '||LEFT_PARANTHESES||','||RIGHT_PARANTHESES||',
                 '||QUOTATION_MARK||',
                 '||SEMICOLON||',
                 '||DASH||',
                 '||EXCLAMATION_MARK||',
                 '||RETURN||']
    
    dict_tokens=dict(zip(list_symbols,list_tokens))    
    #print(dict_tokens)
    
    return dict_tokens

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Pre-process all the data and save it

Running the code cell below will pre-process all the data and save it to file. You're encouraged to lok at the code for `preprocess_and_save_data` in the `helpers.py` file to see what it's doing in detail, but you do not need to change this code.

In [12]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# pre-process training data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [13]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

In [14]:
#print(int_to_vocab)

## Build the Neural Network
In this section, you'll build the components necessary to build an RNN by implementing the RNN Module and forward and backpropagation functions.

### Check Access to GPU

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch

# Check for a GPU
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('No GPU found. Please use a GPU to train your neural network.')

## Input
Let's start with the preprocessed input data. We'll use [TensorDataset](http://pytorch.org/docs/master/data.html#torch.utils.data.TensorDataset) to provide a known format to our dataset; in combination with [DataLoader](http://pytorch.org/docs/master/data.html#torch.utils.data.DataLoader), it will handle batching, shuffling, and other dataset iteration functions.

You can create data with TensorDataset by passing in feature and target tensors. Then create a DataLoader as usual.
```
data = TensorDataset(feature_tensors, target_tensors)
data_loader = torch.utils.data.DataLoader(data, 
                                          batch_size=batch_size)
```

### Batching
Implement the `batch_data` function to batch `words` data into chunks of size `batch_size` using the `TensorDataset` and `DataLoader` classes.

>You can batch words using the DataLoader, but it will be up to you to create `feature_tensors` and `target_tensors` of the correct size and content for a given `sequence_length`.

For example, say we have these as input:
```
words = [1, 2, 3, 4, 5, 6, 7]
sequence_length = 4
```

Your first `feature_tensor` should contain the values:
```
[1, 2, 3, 4]
```
And the corresponding `target_tensor` should just be the next "word"/tokenized word value:
```
5
```
This should continue with the second `feature_tensor`, `target_tensor` being:
```
[2, 3, 4, 5]  # features
6             # target
```

In [16]:
from torch.utils.data import TensorDataset, DataLoader

def batch_data(words, sequence_length, batch_size):
    """
    Batch the neural network data using DataLoader
    :param words: The word ids of the TV scripts
    :param sequence_length: The sequence length of each batch
    :param batch_size: The size of each batch; the number of sequences in a batch
    :return: DataLoader with batched data
    """
    # TODO 3: Implement function
    
    #Transform parameters
    batch_size=int(batch_size)
    seq_length=int(sequence_length)
    arr=words
    
    #Calculate batch size
    batch_size_total=int(seq_length*batch_size)
    #print('Batch size total: ',batch_size_total)

    #Calculate number of batches
    n_batches=len(arr)//batch_size_total
    #print('Number Batches: ',n_batches)
    
    last_char=int((n_batches*batch_size_total))
    arr_x_cut=np.array(arr[:last_char])
    
    x_words, y_targets = [], []
    for i in range(0, len(arr_x_cut)-sequence_length):
        x_batch = arr_x_cut[i:i+sequence_length]
        x_words.append(x_batch)
        y_batch = arr_x_cut[i+sequence_length]
        y_targets.append(y_batch)
    
    #print('Shape Array Features: ',x_batch.shape)
    #print('Shape Array Targets: ',y_batch.shape)
    #print('Shape Array Targets: ',arr_y.shape)
    
    #Transform arrays to tensors
    feature_tensors=torch.from_numpy(np.array(x_words))
    target_tensors=torch.from_numpy(np.array(y_targets))
    
    #Create dataloaders
    data = TensorDataset(feature_tensors, target_tensors)
    data_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)
    
    #return data_loader
    return data_loader

# there is no test for this function, but you are encouraged to create
# print statements and tests of your own

### Test your dataloader 

You'll have to modify this code to test a batching function, but it should look fairly similar.

Below, we're generating some test text data and defining a dataloader using the function you defined, above. Then, we are getting some sample batch of inputs `sample_x` and targets `sample_y` from our dataloader.

Your code should return something like the following (likely in a different order, if you shuffled your data):

```
torch.Size([10, 5])
tensor([[ 28,  29,  30,  31,  32],
        [ 21,  22,  23,  24,  25],
        [ 17,  18,  19,  20,  21],
        [ 34,  35,  36,  37,  38],
        [ 11,  12,  13,  14,  15],
        [ 23,  24,  25,  26,  27],
        [  6,   7,   8,   9,  10],
        [ 38,  39,  40,  41,  42],
        [ 25,  26,  27,  28,  29],
        [  7,   8,   9,  10,  11]])

torch.Size([10])
tensor([ 33,  26,  22,  39,  16,  28,  11,  43,  30,  12])
```

### Sizes
Your sample_x should be of size `(batch_size, sequence_length)` or (10, 5) in this case and sample_y should just have one dimension: batch_size (10). 

### Values

You should also notice that the targets, sample_y, are the *next* value in the ordered test_text data. So, for an input sequence `[ 28,  29,  30,  31,  32]` that ends with the value `32`, the corresponding output should be `33`.

In [17]:
# test dataloader

test_text = range(50)
t_loader = batch_data(test_text, sequence_length=5, batch_size=10)

data_iter = iter(t_loader)
sample_x, sample_y = data_iter.next()

print(sample_x.shape)
print(sample_x)
print()
print(sample_y.shape)
print(sample_y)

torch.Size([10, 5])
tensor([[  0,   1,   2,   3,   4],
        [  1,   2,   3,   4,   5],
        [  2,   3,   4,   5,   6],
        [  3,   4,   5,   6,   7],
        [  4,   5,   6,   7,   8],
        [  5,   6,   7,   8,   9],
        [  6,   7,   8,   9,  10],
        [  7,   8,   9,  10,  11],
        [  8,   9,  10,  11,  12],
        [  9,  10,  11,  12,  13]])

torch.Size([10])
tensor([  5,   6,   7,   8,   9,  10,  11,  12,  13,  14])


---
## Build the Neural Network
Implement an RNN using PyTorch's [Module class](http://pytorch.org/docs/master/nn.html#torch.nn.Module). You may choose to use a GRU or an LSTM. To complete the RNN, you'll have to implement the following functions for the class:
 - `__init__` - The initialize function. 
 - `init_hidden` - The initialization function for an LSTM/GRU hidden state
 - `forward` - Forward propagation function.
 
The initialize function should create the layers of the neural network and save them to the class. The forward propagation function will use these layers to run forward propagation and generate an output and a hidden state.

**The output of this model should be the *last* batch of word scores** after a complete sequence has been processed. That is, for each input sequence of words, we only want to output the word scores for a single, most likely, next word.

### Hints

1. Make sure to stack the outputs of the lstm to pass to your fully-connected layer, you can do this with `lstm_output = lstm_output.contiguous().view(-1, self.hidden_dim)`
2. You can get the last batch of word scores by shaping the output of the final, fully-connected layer like so:

```
# reshape into (batch_size, seq_length, output_size)
output = output.view(batch_size, -1, self.output_size)
# get last batch
out = output[:, -1]
```

In [20]:
import torch.nn as nn

class RNN(nn.Module):
    
#TODO 4 Create RNN
    
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5):
        """
        Initialize the PyTorch RNN Module
        :param vocab_size: The number of input dimensions of the neural network (the size of the vocabulary)
        :param output_size: The number of output dimensions of the neural network
        :param embedding_dim: The size of embeddings, should you choose to use them        
        :param hidden_dim: The size of the hidden layer outputs
        :param dropout: dropout to add in between LSTM/GRU layers
        """
        super(RNN, self).__init__()

        # Embedding and LSTM layers
        self.embed=nn.Embedding(vocab_size,embedding_dim)
        self.lstm=nn.LSTM(embedding_dim,hidden_dim, n_layers, dropout=dropout,batch_first=True)
        
        # Linear and Sigmoid layer
        self.fc=nn.Linear(hidden_dim, output_size)
        
        # RNN base layers
        self.output_size=output_size
        self.n_layers=n_layers
        self.hidden_dim=hidden_dim
    
    def forward(self, nn_input, hidden):
        """
        Forward propagation of the neural network
        :param nn_input: The input to the neural network
        :param hidden: The hidden state        
        :return: Two Tensors, the output of the neural network and the latest hidden state
        """
        batch_size=nn_input.size(0)
        #print('batch size: ',batch_size)
        #print('hidden dim: ',self.hidden_dim)
        #print('output size: ',self.output_size)
        #print('embedding dimension: ', self.embedding_dim)
        #print('vocab size: ',self.vocab_size)
        #print('hidden: ',hidden.size(0))
        
        # embedding  
        nn_input=self.embed(nn_input)
        #print('nn_input after embedding: ',nn_input.shape)
        
        # lstm
        lstm_out,hidden=self.lstm(nn_input,hidden)
        
        # stack lstm cells
        lstm_out=lstm_out.contiguous().view(-1,self.hidden_dim)
        #print('lstm_out: ',lstm_out.shape)
        
        # fully connected layers
        lstm_out=self.fc(lstm_out)
        #print('output shape after fc: ',lstm_out.shape)
        
        # return one batch of output word scores and the hidden state
        lstm_out=lstm_out.view(batch_size,-1,self.output_size)
        #print('output shape after reshape I: ',lstm_out.shape)
        lstm_out = lstm_out[:, -1]
        #print('output shape after reshape II: ',lstm_out.shape)
        
        return lstm_out,hidden
    
    def init_hidden(self, batch_size):
        '''
        Initialize the hidden state of an LSTM/GRU
        :param batch_size: The batch_size of the hidden state
        :return: hidden state of dims (n_layers, batch_size, hidden_dim)
        '''
        # Implement function
        weight=next(self.parameters()).data
        
        if train_on_gpu:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_(),
                      weight.new(self.n_layers, batch_size, self.hidden_dim).zero_())
        # initialize hidden state with zero weights, and move to GPU if available
        
        return hidden

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_rnn(RNN, train_on_gpu)

Tests Passed


### Define forward and backpropagation

Use the RNN class you implemented to apply forward and back propagation. This function will be called, iteratively, in the training loop as follows:
```
loss = forward_back_prop(decoder, decoder_optimizer, criterion, inp, target)
```

And it should return the average loss over a batch and the hidden state returned by a call to `RNN(inp, hidden)`. Recall that you can get this loss by computing it, as usual, and calling `loss.item()`.

**If a GPU is available, you should move your data to that GPU device, here.**

In [21]:
import utils

def forward_back_prop(rnn, optimizer, criterion, inp, target, hidden):
    """
    Forward and backward propagation on the neural network
    :param decoder: The PyTorch Module that holds the neural network
    :param decoder_optimizer: The PyTorch optimizer for the neural network
    :param criterion: The PyTorch loss function
    :param inp: A batch of input to the neural network
    :param target: The target output for the batch of input
    :return: The loss and the latest hidden state Tensor
    """
    
    # TODO 5: Implement Function
    
    # move data to GPU, if available
    if train_on_gpu:
        inp, target = inp.cuda(), target.cuda()
    
    # init hidden state
    hidden=tuple([each.data for each in hidden])
    #h = rnn.init_hidden(64)
    
    # clear gradients
    if train_on_gpu:
        rnn.cuda()
    
    rnn.zero_grad()
    
    # forward pass
    output,hidden=rnn(inp,hidden)
    
    # calculate loss
    loss=criterion(output,target)
    
    # perform backpropagation
    loss.backward()
    
    # clip gradient
    clip=5
    nn.utils.clip_grad_norm_(rnn.parameters(), clip)
    
    # update
    optimizer.step()

    # return the loss over a batch and the hidden state produced by our model
    return loss.item(),hidden

# Note that these tests aren't completely extensive.
# they are here to act as general checks on the expected outputs of your functions
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_forward_back_prop(RNN, forward_back_prop, train_on_gpu)

Tests Passed


## Neural Network Training

With the structure of the network complete and data ready to be fed in the neural network, it's time to train it.

### Train Loop

The training loop is implemented for you in the `train_decoder` function. This function will train the network over all the batches for the number of epochs given. The model progress will be shown every number of batches. This number is set with the `show_every_n_batches` parameter. You'll set this parameter along with other parameters in the next section.

In [33]:
import matplotlib.pyplot as plt
%matplotlib inline
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

def train_rnn(rnn, batch_size, optimizer, criterion, n_epochs, show_every_n_batches=100):
    batch_losses = []
    #loss_dev=[]
    #loss_step=[]
    
    rnn.train()

    print("Training for %d epoch(s)..." % n_epochs)
    
    step=0

    #with active_session():
    # do long-running work here
    
    for epoch_i in keep_awake(range(1, n_epochs + 1)):
            
        #Added two lists to track training progress by matplotlib visuals
        #loss_dev=[]
        #loss_step=[]
        # initialize hidden state
        hidden = rnn.init_hidden(batch_size)

        for batch_i, (inputs, labels) in enumerate(train_loader, 1):

                # make sure you iterate over completely full batches, only
            n_batches = len(train_loader.dataset)//batch_size
            if(batch_i > n_batches):
                break

                # forward, back prop
            loss, hidden = forward_back_prop(rnn, optimizer, criterion, inputs, labels, hidden)          
                # record loss
            batch_losses.append(loss)
            step+=1
            #loss_dev.append(loss)
            #loss_step.append(step)

            # printing loss stats
            if batch_i % show_every_n_batches == 0:
                print('Epoch: {:>4}/{:<4}  Loss: {}\n'.format(
                epoch_i, n_epochs, np.average(batch_losses)))
                batch_losses = []
    
    # returns a trained rnn
    #return rnn, loss_dev, loss_step
    return rnn

### Hyperparameters

Set and train the neural network with the following parameters:
- Set `sequence_length` to the length of a sequence.
- Set `batch_size` to the batch size.
- Set `num_epochs` to the number of epochs to train for.
- Set `learning_rate` to the learning rate for an Adam optimizer.
- Set `vocab_size` to the number of uniqe tokens in our vocabulary.
- Set `output_size` to the desired size of the output.
- Set `embedding_dim` to the embedding dimension; smaller than the vocab_size.
- Set `hidden_dim` to the hidden dimension of your RNN.
- Set `n_layers` to the number of layers/cells in your RNN.
- Set `show_every_n_batches` to the number of batches at which the neural network should print progress.

If the network isn't getting the desired results, tweak these parameters and/or the layers in the `RNN` class.

In [34]:
# TODO 6: Data params
# Sequence Length
sequence_length = 10# 5 # of words in a sequence
# Batch Size
batch_size = 64

# data loader - do not change
train_loader = batch_data(int_text, sequence_length, batch_size)

In [35]:
#print(len(int_to_vocab))

In [36]:
#print(len(int_text))

In [37]:
# TODO 7: Training parameters
# Number of Epochs
num_epochs = 15
# Learning Rate
learning_rate = 0.001#0.00075

# Model parameters
# Vocab size
vocab_size = len(int_to_vocab)
# Output size
output_size = vocab_size
# Embedding Dimension
embedding_dim = 128
# Hidden Dimension
hidden_dim = 256
# Number of RNN Layers
n_layers = 3

# Show stats for every n number of batches
show_every_n_batches = 500

### Train
In the next cell, you'll train the neural network on the pre-processed data.  If you have a hard time getting a good loss, you may consider changing your hyperparameters. In general, you may get better results with larger hidden and n_layer dimensions, but larger models take a longer time to train. 
> **You should aim for a loss less than 3.5.** 

You should also experiment with different sequence lengths, which determine the size of the long range dependencies that a model can learn.

In [38]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""

# create model and move to gpu if available
rnn = RNN(vocab_size, output_size, embedding_dim, hidden_dim, n_layers, dropout=0.5)
if train_on_gpu:
    rnn.cuda()

# defining loss and optimization functions for training
optimizer = torch.optim.Adam(rnn.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

# training the model
trained_rnn = train_rnn(rnn, batch_size, optimizer, criterion, num_epochs, show_every_n_batches)

# saving the trained model
helper.save_model('./save/trained_rnn2', trained_rnn)
print('Model Trained and Saved')

Training for 15 epoch(s)...
Epoch:    1/15    Loss: 5.900190586566925

Epoch:    1/15    Loss: 5.7647614679336545

Epoch:    1/15    Loss: 5.619286236286164

Epoch:    1/15    Loss: 5.04694272851944

Epoch:    1/15    Loss: 4.9191392765045165

Epoch:    1/15    Loss: 4.756851031780243

Epoch:    1/15    Loss: 4.672598716259003

Epoch:    1/15    Loss: 4.721780804157257

Epoch:    1/15    Loss: 4.758245852947235

Epoch:    1/15    Loss: 4.609064590454102

Epoch:    1/15    Loss: 4.701595052242279

Epoch:    1/15    Loss: 4.72786857843399

Epoch:    1/15    Loss: 4.65733225107193

Epoch:    1/15    Loss: 4.580588966369629

Epoch:    1/15    Loss: 4.591768788814544

Epoch:    1/15    Loss: 4.394650734901428

Epoch:    1/15    Loss: 4.419550692081452

Epoch:    1/15    Loss: 4.492304069519043

Epoch:    1/15    Loss: 4.441064915180206

Epoch:    1/15    Loss: 4.369892109870911

Epoch:    1/15    Loss: 4.4802052369117735

Epoch:    1/15    Loss: 4.554030413150787

Epoch:    1/15    Loss: 4.

Epoch:    8/15    Loss: 3.6353931034342724

Epoch:    8/15    Loss: 3.5684031095504762

Epoch:    8/15    Loss: 3.611725805282593

Epoch:    8/15    Loss: 3.5421569280624388

Epoch:    8/15    Loss: 3.56511793756485

Epoch:    8/15    Loss: 3.492931531429291

Epoch:    8/15    Loss: 3.4794641637802126

Epoch:    8/15    Loss: 3.5265471835136415

Epoch:    8/15    Loss: 3.5744054675102235

Epoch:    8/15    Loss: 3.513272617816925

Epoch:    8/15    Loss: 3.6192163710594176

Epoch:    8/15    Loss: 3.6256602268218994

Epoch:    8/15    Loss: 3.5899871010780333

Epoch:    8/15    Loss: 3.552377757549286

Epoch:    8/15    Loss: 3.5482976570129394

Epoch:    8/15    Loss: 3.478928514480591

Epoch:    8/15    Loss: 3.4978986802101137

Epoch:    8/15    Loss: 3.549002081871033

Epoch:    8/15    Loss: 3.5216948437690734

Epoch:    8/15    Loss: 3.462303900718689

Epoch:    8/15    Loss: 3.6129317560195924

Epoch:    8/15    Loss: 3.6526301345825196

Epoch:    8/15    Loss: 3.69347097826004


Epoch:   15/15    Loss: 3.408419171333313

Epoch:   15/15    Loss: 3.4235182859897613

Epoch:   15/15    Loss: 3.3807004985809326

Epoch:   15/15    Loss: 3.3889014525413512

Epoch:   15/15    Loss: 3.3384930675029754

Epoch:   15/15    Loss: 3.320569023609161

Epoch:   15/15    Loss: 3.353462830543518

Epoch:   15/15    Loss: 3.385305607318878

Epoch:   15/15    Loss: 3.3434659180641173

Epoch:   15/15    Loss: 3.4266078815460204

Epoch:   15/15    Loss: 3.4526493306159973

Epoch:   15/15    Loss: 3.4380586910247803

Epoch:   15/15    Loss: 3.3929636936187744

Epoch:   15/15    Loss: 3.3814966049194335

Epoch:   15/15    Loss: 3.3139064185619356

Epoch:   15/15    Loss: 3.3127105703353883

Epoch:   15/15    Loss: 3.364194236755371

Epoch:   15/15    Loss: 3.3545953979492187

Epoch:   15/15    Loss: 3.3048807566165923

Epoch:   15/15    Loss: 3.4286859946250914

Epoch:   15/15    Loss: 3.451382818222046

Epoch:   15/15    Loss: 3.495078672885895

Epoch:   15/15    Loss: 3.4592545719146

/opt/conda/lib/python3.6/site-packages/torch/serialization.py:193: UserWarning: Couldn't retrieve source code for container of type RNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Model Trained and Saved


### Question: How did you decide on your model hyperparameters? 
For example, did you try different sequence_lengths and find that one size made the model converge faster? What about your hidden_dim and n_layers; how did you decide on those?

**Answer:** 

**Started with the following parameters** which I got from recommendations out of the video tutorials and by reading blog posts in the internet

Start Parameters were [lr=0.001, batch_size=32, seq_length=10, n_layers=2, hidden_dim=128, embedding_dim=200].
Loss decreased from 5.8 to 4.1 after 5 epochs. Decided to increase sequence_length by factor 5x to 50.

**Experiment No 2** Parameters were [lr=0.001, batch_size=32, seq_length=50, n_layers=2, hidden_dim=128, embedding_dim=200]. Loss decreased from 5.8 to 4.3 after 5 epochs. Set seq_length back to 10 and tried to increase learning rate by factor 10x to 0.01 in order to speed up training progress.

**Experiment No 3** Parameters were [lr=0.01, batch_size=32, seq_length=10, n_layers=2, hidden_dim=128, embedding_dim=200]. Loss decreased from 5.6 to 5.1 after 5 epochs. Loss values bounced back and forth, deduced that learning rate might be too high to approach global minimum. Decreased learning rate by factor 0.5x to 0.005.

**Experiment No 4** Parameters were [lr=0.005, batch_size=32, seq_length=10, n_layers=2, hidden_dim=128, embedding_dim=200]. Loss decreased from 5.5 to 4.8 after 5 epochs. Loss values bounced again back and forth. Decreased learning rate by factor 0.5x to 0.0025 to reach a more or less constant decrease of loss.

**Experiment No 5** Parameters were [lr=0.00025, batch_size=32, seq_length=10, n_layers=2, hidden_dim=128, embedding_dim=200]. Loss decreased from 5.9 to 4.5 after 5 epochs. Training progresses slowly.

**Experiment No 6** Parameters were [lr=0.001, batch_size=32, seq_length=10, n_layers=2, hidden_dim=128, embedding_dim=200]. Loss decreased from 5.9 to 4.5 after 5 epochs. Increased number of layers to 4 and hidden_dim to 256 to enlargen the network and thus provide more degrees of freedom to reach a valueable fit.

**Experiment No 7** Parameters were [lr=0.001, batch_size=32, seq_length=10, n_layers=4, hidden_dim=256, embedding_dim=200]. No real progress detected. Increased number of layers to 7.

**Experiment No 8** Parameters were [lr=0.001, batch_size=32, seq_length=10, n_layers=7, hidden_dim=256, embedding_dim=200]. No real progress detected, however training speed decreased. Decreased number of layers to 4 again. Next, focused on embedding_dim as parameter, reduced size of embedding_dim from 200 to 100.

**Experiment No 9** Parameters were [lr=0.001, batch_size=32, seq_length=10, n_layers=4, hidden_dim=256, embedding_dim=100]. Progress! Loss decreased to 3.9 after 5 epochs, less bouncing. Reduced size of embedding_dim further from 100 to 50.

**Experiment No 10** Parameters were [lr=0.001, batch_size=32, seq_length=10, n_layers=4, hidden_dim=256, embedding_dim=50]. No progress detected. Loss decreased to 3.9 after 5 epochs, still less bouncing. Increased size of embedding_dim to 100 again. Decreased learning rate to 0.0005 to reach more continuity.

**Experiment No 11** Parameters were [lr=0.0005, batch_size=32, seq_length=10, n_layers=4, hidden_dim=256, embedding_dim=100]. Progress is shown by stable loss decrease, however final loss is still around 3.8 after 5 epochs. Increased batch size to 64.

**Experiment No 12** Parameters were [lr=0.0005, batch_size=64, seq_length=10, n_layers=4, hidden_dim=256, embedding_dim=100]. No progress detected. Increased size of hidden_dim to 512.

**Experiment No 13** Parameters were [lr=0.0005, batch_size=64, seq_length=10, n_layers=4, hidden_dim=512, embedding_dim=100]. Training slows down significantly. Chose next parameter set by using intuition and
experiences from former tests. 

**Experiment No 14** Parameters were [lr=0.00075, batch_size=32, seq_length=10, n_layers=3, hidden_dim=256, embedding_dim=100]. Training loss decreases very rapidly in the first epoch and then constantly by 0.1 from
epoch to epoch. Reached a loss of 3.3-3.4 after 12 epochs (Could have trained even more and might have got even better performance, no saturation effects were visible)!

**Experiment No 15 (second submission)** Parameters were [lr=0.001, batch_size=32, seq_length=10, n_layers=3, hidden_dim=256, embedding_dim=100]. Increased learning rate due to issues with saving/loading the model. Training loss decreases also rapidly and constantly. Reached a loss of 3.3-3.4 after 15 epochs (Could have trained even more and might have got even better performance, no saturation effects were visible)!

---
# Checkpoint

After running the above training cell, your model will be saved by name, `trained_rnn`, and if you save your notebook progress, **you can pause here and come back to this code at another time**. You can resume your progress by running the next cell, which will load in our word:id dictionaries _and_ load in your saved model by name!

In [ ]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import torch
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
trained_rnn = helper.load_model('./save/trained_rnn2')

In [ ]:
print(trained_rnn)

## Generate TV Script
With the network trained and saved, you'll use it to generate a new, "fake" Seinfeld TV script in this section.

### Generate Text
To generate the text, the network needs to start with a single word and repeat its predictions until it reaches a set length. You'll be using the `generate` function to do this. It takes a word id to start with, `prime_id`, and generates a set length of text, `predict_len`. Also note that it uses topk sampling to introduce some randomness in choosing the most likely next word, given an output set of word scores!

In [39]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import torch.nn.functional as F

def generate(rnn, prime_id, int_to_vocab, token_dict, pad_value, predict_len=100):
    """
    Generate text using the neural network
    :param decoder: The PyTorch Module that holds the trained neural network
    :param prime_id: The word id to start the first prediction
    :param int_to_vocab: Dict of word id keys to word values
    :param token_dict: Dict of puncuation tokens keys to puncuation values
    :param pad_value: The value used to pad a sequence
    :param predict_len: The length of text to generate
    :return: The generated text
    """
    rnn.eval()
    
    # create a sequence (batch_size=1) with the prime_id
    current_seq = np.full((1, sequence_length), pad_value)
    current_seq[-1][-1] = prime_id
    predicted = [int_to_vocab[prime_id]]
    
    for _ in range(predict_len):
        if train_on_gpu:
            current_seq = torch.LongTensor(current_seq).cuda()
        else:
            current_seq = torch.LongTensor(current_seq)
        
        # initialize the hidden state
        hidden = rnn.init_hidden(current_seq.size(0))
        
        # get the output of the rnn
        output, _ = rnn(current_seq, hidden)
        
        # get the next word probabilities
        p = F.softmax(output, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
         
        # use top_k sampling to get the index of the next word
        top_k = 5
        p, top_i = p.topk(top_k)
        top_i = top_i.numpy().squeeze()
        
        # select the likely next word index with some element of randomness
        p = p.numpy().squeeze()
        word_i = np.random.choice(top_i, p=p/p.sum())
        
        # retrieve that word from the dictionary
        word = int_to_vocab[word_i]
        predicted.append(word)     
        
        # the generated word becomes the next "current sequence" and the cycle can continue
        current_seq = np.roll(current_seq, -1, 1)
        current_seq[-1][-1] = word_i
    
    gen_sentences = ' '.join(predicted)
    
    # Replace punctuation tokens
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_sentences = gen_sentences.replace(' ' + token.lower(), key)
    gen_sentences = gen_sentences.replace('\n ', '\n')
    gen_sentences = gen_sentences.replace('( ', '(')
    
    # return all the sentences
    return gen_sentences

### Generate a New Script
It's time to generate the text. Set `gen_length` to the length of TV script you want to generate and set `prime_word` to one of the following to start the prediction:
- "jerry"
- "elaine"
- "george"
- "kramer"

You can set the prime word to _any word_ in our dictionary, but it's best to start with a name for generating a TV script. (You can also start with any other names you find in the original text file!)

In [48]:
# run the cell multiple times to get different results!
gen_length = 400 # modify the length to your preference
prime_word = 'george' # name for starting the script

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
pad_word = helper.SPECIAL_WORDS['PADDING']
generated_script = generate(trained_rnn, vocab_to_int[prime_word + ':'], int_to_vocab, token_dict, vocab_to_int[pad_word], gen_length)
print(generated_script)

george: a frog.

elaine: oh. i think you were.

helen: i think i was just going to get the subway. you know how i was in a mood, and i can get this.

helen: oh, no.

kramer: no. you didn't even have to be humiliated.

morty: you know, i was wondering, i can't go.

jerry: so?

jerry: well, i don't want to see the bathrooms.

jerry: what about the bucket of that?

george: i thought you can prove.

jerry: well, i think you could get a lot.

elaine: well, you know, i think i was a little tired about this.

george: oh, come on.

george: you know, i can't find this.

george: oh..

george: what do i mean?

george: what is it?

jerry: what is the matter with that thing?

jerry: no.

george: oh..

elaine: i can't believe i can.

jerry: i thought it was a little funny.

helen: what is that?

jerry: i can't get out.

helen: what?

elaine: yeah, it's a new awkward code.

elaine: well, you should get a little bit.

elaine: oh, i forgot to find it.

kramer: well, i guess we should do this.

helen: i

#### Save your favorite scripts

Once you have a script that you like (or find interesting), save it to a text file!

In [41]:
# save script to a text file
f =  open("generated_script_1.txt","w")
f.write(generated_script)
f.close()

# The TV Script is Not Perfect
It's ok if the TV script doesn't make perfect sense. It should look like alternating lines of dialogue, here is one such example of a few generated lines.

### Example generated script

>jerry: what about me?
>
>jerry: i don't have to wait.
>
>kramer:(to the sales table)
>
>elaine:(to jerry) hey, look at this, i'm a good doctor.
>
>newman:(to elaine) you think i have no idea of this...
>
>elaine: oh, you better take the phone, and he was a little nervous.
>
>kramer:(to the phone) hey, hey, jerry, i don't want to be a little bit.(to kramer and jerry) you can't.
>
>jerry: oh, yeah. i don't even know, i know.
>
>jerry:(to the phone) oh, i know.
>
>kramer:(laughing) you know...(to jerry) you don't know.

You can see that there are multiple characters that say (somewhat) complete sentences, but it doesn't have to be perfect! It takes quite a while to get good results, and often, you'll have to use a smaller vocabulary (and discard uncommon words), or get more data.  The Seinfeld dataset is about 3.4 MB, which is big enough for our purposes; for script generation you'll want more than 1 MB of text, generally. 

# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save another copy as an HTML file by clicking "File" -> "Download as.."->"html". Include the "helper.py" and "problem_unittests.py" files in your submission. Once you download these files, compress them into one zip file for submission.